In [ ]:
#The model is recognizing fresh and rotten fruits. The dataset comes from Kaggle.
#The dataset structure is in the fruits folder. There are 6 categories of fruits: fresh apples, fresh oranges, 
#fresh bananas, rotten apples, rotten oranges, and rotten bananas.

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
base_model.trainable = False  # Freezing base model

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

x = keras.layers.GlobalAveragePooling2D()(x) 

outputs = keras.layers.Dense(6, activation = 'softmax')(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy' , metrics = ["accuracy"]) # Categorical crossentropy is needed as we have more than 2 categories

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,  
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    horizontal_flip=True,
    vertical_flip=True)

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('fruits/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('fruits/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)


#model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

In [ ]:
#Optional step for increasing model's accuracy

base_model.trainable = True  # Unfreeze the base model

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = 'categorical_crossentropy', metrics = ["accuracy"])

model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=10)

In [ ]:
#Model's evaluation

model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)